In [65]:
from scipy.stats import rankdata
import scipy as sp
import numpy as np
import pandas as pd

# 读取数据

In [66]:
df = pd.read_csv('./alldata.csv')

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [128]:
def get_data(column):
    df_res = df.pivot(index=['Trddt'], columns='Stkcd', values=[column])
    df_res = df_res[column]
    df_res = df_res.reset_index()
    df_res.fillna(df_res.mean(), inplace=True)
    df_res = df_res.set_index('Trddt')
    return df_res
                    

### 开盘价因子

In [67]:
df_open = df.pivot(index=['Trddt'], columns='Stkcd', values=['Opnprc'])
df_open

Opnprc                                                          \
Stkcd      1      2      4      5      6      7      8      9      10       
Trddt                                                                       
2010-01-04  24.52  10.85    NaN   6.01  11.33   7.09    NaN  11.00  11.95   
2010-01-05  23.75  10.51    NaN   5.95  11.12   6.94    NaN  10.89  11.81   
2010-01-06  23.25  10.35    NaN   6.13  10.79   7.17    NaN  10.56  11.61   
2010-01-07  22.90  10.36    NaN   6.03  10.85   7.16    NaN  10.44  11.73   
2010-01-08  22.50  10.28    NaN   5.80  10.70   6.94    NaN  10.70  11.15   
...           ...    ...    ...    ...    ...    ...    ...    ...    ...   
2022-01-24  17.34  21.59  21.00   2.10   4.50   7.18   2.63  14.00   3.70   
2022-01-25  17.08  21.60  21.10   2.11   4.47   6.93   2.65  14.28   3.59   
2022-01-26  16.95  20.99  22.26   2.05   4.31   6.88   2.56  13.61   3.40   
2022-01-27  16.50  20.53  23.00   2.13   4.33   6.51   2.56  13.29   3.43   
2022-01-28  16.39  20.60  20.00   2.07   4.26   6.29   2.52  12.99   3.35   

                   ...                                                   \
Stkcd      11      ... 900947 900948 900949 900950 900951 900952 900953   
Trddt              ...                                                    
2010-01-04  10.79  ...  0.843  8.660  0.708  0.837  0.541  0.484  0.727   
2010-01-05  10.60  ...  0.837  8.758  0.697  0.809  0.548  0.481  0.720   
2010-01-06  10.27  ...  0.840  9.019  0.704  0.818  0.540  0.485  0.739   
2010-01-07  10.18  ...  0.835  8.936  0.700  0.830  0.537  0.481  0.751   
2010-01-08  10.01  ...  0.822  8.770  0.693  0.826  0.528  0.475  0.745   
...           ...  ...    ...    ...    ...    ...    ...    ...    ...   
2022-01-24  11.49  ...  0.282  0.905    NaN    NaN    NaN  0.246  0.334   
2022-01-25  11.42  ...  0.278  0.908    NaN    NaN    NaN  0.244  0.333   
2022-01-26  10.96  ...  0.272  0.882    NaN    NaN    NaN  0.244  0.328   
2022-01-27  11.11  ...  0.271  0.892    NaN    NaN    NaN  0.247  0.322   
2022-01-28  10.86  ...  0.271  0.915    NaN    NaN    NaN  0.245  0.326   

                                 
Stkcd      900955 900956 900957  
Trddt                            
2010-01-04  0.716  0.812  0.559  
2010-01-05  0.703  0.801  0.548  
2010-01-06  0.695  0.813  0.548  
2010-01-07  0.709  0.806  0.542  
2010-01-08  0.705  0.790  0.527  
...           ...    ...    ...  
2022-01-24  0.120    NaN  0.633  
2022-01-25  0.117    NaN  0.617  
2022-01-26  0.116    NaN  0.614  
2022-01-27  0.113    NaN  0.610  
2022-01-28  0.111    NaN  0.615  

[2936 rows x 4824 columns]

In [68]:
df_open = df_open['Opnprc']
df_open

Stkcd,1,2,4,5,6,7,8,9,10,11,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
Trddt,,,,,,,,,,,,,,,,,,,,,
2010-01-04,24.52,10.85,NaN,6.01,11.33,7.09,NaN,11.00,11.95,10.79,...,0.843,8.660,0.708,0.837,0.541,0.484,0.727,0.716,0.812,0.559
2010-01-05,23.75,10.51,NaN,5.95,11.12,6.94,NaN,10.89,11.81,10.60,...,0.837,8.758,0.697,0.809,0.548,0.481,0.720,0.703,0.801,0.548
2010-01-06,23.25,10.35,NaN,6.13,10.79,7.17,NaN,10.56,11.61,10.27,...,0.840,9.019,0.704,0.818,0.540,0.485,0.739,0.695,0.813,0.548
2010-01-07,22.90,10.36,NaN,6.03,10.85,7.16,NaN,10.44,11.73,10.18,...,0.835,8.936,0.700,0.830,0.537,0.481,0.751,0.709,0.806,0.542
2010-01-08,22.50,10.28,NaN,5.80,10.70,6.94,NaN,10.70,11.15,10.01,...,0.822,8.770,0.693,0.826,0.528,0.475,0.745,0.705,0.790,0.527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,17.34,21.59,21.00,2.10,4.50,7.18,2.63,14.00,3.70,11.49,...,0.282,0.905,NaN,NaN,NaN,0.246,0.334,0.120,NaN,0.633
2022-01-25,17.08,21.60,21.10,2.11,4.47,6.93,2.65,14.28,3.59,11.42,...,0.278,0.908,NaN,NaN,NaN,0.244,0.333,0.117,NaN,0.617
2022-01-26,16.95,20.99,22.26,2.05,4.31,6.88,2.56,13.61,3.40,10.96,...,0.272,0.882,NaN,NaN,NaN,0.244,0.328,0.116,NaN,0.614


In [69]:
df_open = df_open.reset_index()

In [70]:
df_open.columns

Index(['Trddt',       1,       2,       4,       5,       6,       7,       8,
             9,      10,
       ...
        900947,  900948,  900949,  900950,  900951,  900952,  900953,  900955,
        900956,  900957],
      dtype='object', name='Stkcd', length=4825)

In [71]:
df_open

Stkcd,Trddt,1,2,4,5,6,7,8,9,10,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
0,2010-01-04,24.52,10.85,NaN,6.01,11.33,7.09,NaN,11.00,11.95,...,0.843,8.660,0.708,0.837,0.541,0.484,0.727,0.716,0.812,0.559
1,2010-01-05,23.75,10.51,NaN,5.95,11.12,6.94,NaN,10.89,11.81,...,0.837,8.758,0.697,0.809,0.548,0.481,0.720,0.703,0.801,0.548
2,2010-01-06,23.25,10.35,NaN,6.13,10.79,7.17,NaN,10.56,11.61,...,0.840,9.019,0.704,0.818,0.540,0.485,0.739,0.695,0.813,0.548
3,2010-01-07,22.90,10.36,NaN,6.03,10.85,7.16,NaN,10.44,11.73,...,0.835,8.936,0.700,0.830,0.537,0.481,0.751,0.709,0.806,0.542
4,2010-01-08,22.50,10.28,NaN,5.80,10.70,6.94,NaN,10.70,11.15,...,0.822,8.770,0.693,0.826,0.528,0.475,0.745,0.705,0.790,0.527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2931,2022-01-24,17.34,21.59,21.00,2.10,4.50,7.18,2.63,14.00,3.70,...,0.282,0.905,NaN,NaN,NaN,0.246,0.334,0.120,NaN,0.633
2932,2022-01-25,17.08,21.60,21.10,2.11,4.47,6.93,2.65,14.28,3.59,...,0.278,0.908,NaN,NaN,NaN,0.244,0.333,0.117,NaN,0.617
2933,2022-01-26,16.95,20.99,22.26,2.05,4.31,6.88,2.56,13.61,3.40,...,0.272,0.882,NaN,NaN,NaN,0.244,0.328,0.116,NaN,0.614
2934,2022-01-27,16.50,20.53,23.00,2.13,4.33,6.51,2.56,13.29,3.43,...,0.271,0.892,NaN,NaN,NaN,0.247,0.322,0.113,NaN,0.610


In [72]:
df_open.mean(axis=1)

0       15.861567
1       15.874498
2       15.996408
3       15.989290
4       15.596966
          ...    
2931    25.750472
2932    25.899940
2933    25.223211
2934    25.275190
2935    24.647835
Length: 2936, dtype: float64

In [73]:
df_open.mean()

Stkcd
1         14.228678
2         17.624193
4         19.239490
5          4.093732
6          6.571747
            ...    
900952     0.426805
900953     0.642098
900955     0.384115
900956     1.199219
900957     0.696365
Length: 4824, dtype: float64

In [74]:
df_open.fillna(df_open.mean(), inplace=True)

In [75]:
df_open

Stkcd,Trddt,1,2,4,5,6,7,8,9,10,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
0,2010-01-04,24.52,10.85,19.23949,6.01,11.33,7.09,8.723644,11.00,11.95,...,0.843,8.660,0.708000,0.837000,0.541000,0.484,0.727,0.716,0.812000,0.559
1,2010-01-05,23.75,10.51,19.23949,5.95,11.12,6.94,8.723644,10.89,11.81,...,0.837,8.758,0.697000,0.809000,0.548000,0.481,0.720,0.703,0.801000,0.548
2,2010-01-06,23.25,10.35,19.23949,6.13,10.79,7.17,8.723644,10.56,11.61,...,0.840,9.019,0.704000,0.818000,0.540000,0.485,0.739,0.695,0.813000,0.548
3,2010-01-07,22.90,10.36,19.23949,6.03,10.85,7.16,8.723644,10.44,11.73,...,0.835,8.936,0.700000,0.830000,0.537000,0.481,0.751,0.709,0.806000,0.542
4,2010-01-08,22.50,10.28,19.23949,5.80,10.70,6.94,8.723644,10.70,11.15,...,0.822,8.770,0.693000,0.826000,0.528000,0.475,0.745,0.705,0.790000,0.527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2931,2022-01-24,17.34,21.59,21.00000,2.10,4.50,7.18,2.630000,14.00,3.70,...,0.282,0.905,0.592559,0.738369,0.607353,0.246,0.334,0.120,1.199219,0.633
2932,2022-01-25,17.08,21.60,21.10000,2.11,4.47,6.93,2.650000,14.28,3.59,...,0.278,0.908,0.592559,0.738369,0.607353,0.244,0.333,0.117,1.199219,0.617
2933,2022-01-26,16.95,20.99,22.26000,2.05,4.31,6.88,2.560000,13.61,3.40,...,0.272,0.882,0.592559,0.738369,0.607353,0.244,0.328,0.116,1.199219,0.614
2934,2022-01-27,16.50,20.53,23.00000,2.13,4.33,6.51,2.560000,13.29,3.43,...,0.271,0.892,0.592559,0.738369,0.607353,0.247,0.322,0.113,1.199219,0.610


In [76]:
std = df_open.std(axis=1)
mean = df_open.mean(axis=1)
mean

0       27.888514
1       27.884405
2       27.933353
3       27.921725
4       27.781318
          ...    
2931    25.475193
2932    25.606084
2933    24.937605
2934    24.987060
2935    24.367124
Length: 2936, dtype: float64

In [77]:
std

0       39.198982
1       39.210659
2       39.208412
3       39.210787
4       39.210491
          ...    
2931    52.028802
2932    52.392327
2933    52.051452
2934    52.434268
2935    51.397427
Length: 2936, dtype: float64

In [78]:
def change_extreme(row):
    m=row[1:].mean()
    sd=row[1:].std()
    row[1:]=row[1:].apply(lambda x:x if m-3*sd<=x<=m+3*sd else (m+3*sd if x>m+3*sd else m-3*sd))
    return row

In [79]:
df_open.apply(change_extreme, axis=1)

Stkcd,Trddt,1,2,4,5,6,7,8,9,10,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
0,2010-01-04,24.52,10.85,19.23949,6.01,11.33,7.09,8.723644,11.00,11.95,...,0.843,8.660,0.708000,0.837000,0.541000,0.484,0.727,0.716,0.812000,0.559
1,2010-01-05,23.75,10.51,19.23949,5.95,11.12,6.94,8.723644,10.89,11.81,...,0.837,8.758,0.697000,0.809000,0.548000,0.481,0.720,0.703,0.801000,0.548
2,2010-01-06,23.25,10.35,19.23949,6.13,10.79,7.17,8.723644,10.56,11.61,...,0.840,9.019,0.704000,0.818000,0.540000,0.485,0.739,0.695,0.813000,0.548
3,2010-01-07,22.90,10.36,19.23949,6.03,10.85,7.16,8.723644,10.44,11.73,...,0.835,8.936,0.700000,0.830000,0.537000,0.481,0.751,0.709,0.806000,0.542
4,2010-01-08,22.50,10.28,19.23949,5.80,10.70,6.94,8.723644,10.70,11.15,...,0.822,8.770,0.693000,0.826000,0.528000,0.475,0.745,0.705,0.790000,0.527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2931,2022-01-24,17.34,21.59,21.00000,2.10,4.50,7.18,2.630000,14.00,3.70,...,0.282,0.905,0.592559,0.738369,0.607353,0.246,0.334,0.120,1.199219,0.633
2932,2022-01-25,17.08,21.60,21.10000,2.11,4.47,6.93,2.650000,14.28,3.59,...,0.278,0.908,0.592559,0.738369,0.607353,0.244,0.333,0.117,1.199219,0.617
2933,2022-01-26,16.95,20.99,22.26000,2.05,4.31,6.88,2.560000,13.61,3.40,...,0.272,0.882,0.592559,0.738369,0.607353,0.244,0.328,0.116,1.199219,0.614
2934,2022-01-27,16.50,20.53,23.00000,2.13,4.33,6.51,2.560000,13.29,3.43,...,0.271,0.892,0.592559,0.738369,0.607353,0.247,0.322,0.113,1.199219,0.610


In [80]:
df_open

Stkcd,Trddt,1,2,4,5,6,7,8,9,10,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
0,2010-01-04,24.52,10.85,19.23949,6.01,11.33,7.09,8.723644,11.00,11.95,...,0.843,8.660,0.708000,0.837000,0.541000,0.484,0.727,0.716,0.812000,0.559
1,2010-01-05,23.75,10.51,19.23949,5.95,11.12,6.94,8.723644,10.89,11.81,...,0.837,8.758,0.697000,0.809000,0.548000,0.481,0.720,0.703,0.801000,0.548
2,2010-01-06,23.25,10.35,19.23949,6.13,10.79,7.17,8.723644,10.56,11.61,...,0.840,9.019,0.704000,0.818000,0.540000,0.485,0.739,0.695,0.813000,0.548
3,2010-01-07,22.90,10.36,19.23949,6.03,10.85,7.16,8.723644,10.44,11.73,...,0.835,8.936,0.700000,0.830000,0.537000,0.481,0.751,0.709,0.806000,0.542
4,2010-01-08,22.50,10.28,19.23949,5.80,10.70,6.94,8.723644,10.70,11.15,...,0.822,8.770,0.693000,0.826000,0.528000,0.475,0.745,0.705,0.790000,0.527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2931,2022-01-24,17.34,21.59,21.00000,2.10,4.50,7.18,2.630000,14.00,3.70,...,0.282,0.905,0.592559,0.738369,0.607353,0.246,0.334,0.120,1.199219,0.633
2932,2022-01-25,17.08,21.60,21.10000,2.11,4.47,6.93,2.650000,14.28,3.59,...,0.278,0.908,0.592559,0.738369,0.607353,0.244,0.333,0.117,1.199219,0.617
2933,2022-01-26,16.95,20.99,22.26000,2.05,4.31,6.88,2.560000,13.61,3.40,...,0.272,0.882,0.592559,0.738369,0.607353,0.244,0.328,0.116,1.199219,0.614
2934,2022-01-27,16.50,20.53,23.00000,2.13,4.33,6.51,2.560000,13.29,3.43,...,0.271,0.892,0.592559,0.738369,0.607353,0.247,0.322,0.113,1.199219,0.610


In [81]:
df_open_std = df_open.iloc[:,1:].apply(lambda row: (row-row.mean())/row.std(), axis=1)

In [82]:
df_open_std.insert(0, 'Trddt', df_open.iloc[:,0])
df_open_std 

Stkcd,Trddt,1,2,4,5,6,7,8,9,10,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
0,2010-01-04,-0.085934,-0.434667,-0.220644,-0.558140,-0.422422,-0.530588,-0.488912,-0.430841,-0.406605,...,-0.689955,-0.490536,-0.693398,-0.690108,-0.697659,-0.699113,-0.692914,-0.693194,-0.690745,-0.697200
1,2010-01-05,-0.105441,-0.443104,-0.220474,-0.559399,-0.427547,-0.534151,-0.488662,-0.433413,-0.409950,...,-0.689797,-0.487786,-0.693368,-0.690511,-0.697168,-0.698876,-0.692781,-0.693215,-0.690715,-0.697168
2,2010-01-06,-0.119448,-0.448459,-0.221735,-0.556089,-0.437237,-0.529564,-0.489938,-0.443103,-0.416323,...,-0.691009,-0.482405,-0.694477,-0.691570,-0.698660,-0.700063,-0.693585,-0.694707,-0.691697,-0.698456
3,2010-01-07,-0.128070,-0.447880,-0.221425,-0.558309,-0.435383,-0.529490,-0.489612,-0.445840,-0.412941,...,-0.690798,-0.484196,-0.694241,-0.690925,-0.698398,-0.699826,-0.692940,-0.694011,-0.691537,-0.698270
4,2010-01-08,-0.134691,-0.446343,-0.217845,-0.560598,-0.435631,-0.531524,-0.486035,-0.435631,-0.424155,...,-0.687554,-0.484853,-0.690844,-0.687452,-0.695052,-0.696403,-0.689517,-0.690538,-0.688370,-0.695077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2931,2022-01-24,-0.156359,-0.074674,-0.086014,-0.449274,-0.403146,-0.351636,-0.439087,-0.220555,-0.418522,...,-0.484216,-0.472242,-0.478247,-0.475445,-0.477963,-0.484908,-0.483217,-0.487330,-0.466587,-0.477470
2932,2022-01-25,-0.162735,-0.076463,-0.086007,-0.448464,-0.403419,-0.356466,-0.438157,-0.216178,-0.420216,...,-0.483431,-0.471407,-0.477427,-0.474644,-0.477145,-0.484080,-0.482381,-0.486504,-0.465848,-0.476961
2933,2022-01-26,-0.153456,-0.075840,-0.051442,-0.439711,-0.396293,-0.346918,-0.429913,-0.217623,-0.413775,...,-0.473870,-0.462151,-0.467711,-0.464910,-0.467427,-0.474408,-0.472794,-0.476867,-0.456056,-0.467299
2934,2022-01-27,-0.161861,-0.085003,-0.037896,-0.435918,-0.393961,-0.352385,-0.427718,-0.223080,-0.411125,...,-0.471372,-0.459529,-0.465240,-0.462459,-0.464958,-0.471830,-0.470400,-0.474386,-0.453670,-0.464907


In [100]:
df_open_std = df_open_std.set_index('Trddt')
df_open_std

Stkcd,1,2,4,5,6,7,8,9,10,11,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
Trddt,,,,,,,,,,,,,,,,,,,,,
2010-01-04,-0.085934,-0.434667,-0.220644,-0.558140,-0.422422,-0.530588,-0.488912,-0.430841,-0.406605,-0.436198,...,-0.689955,-0.490536,-0.693398,-0.690108,-0.697659,-0.699113,-0.692914,-0.693194,-0.690745,-0.697200
2010-01-05,-0.105441,-0.443104,-0.220474,-0.559399,-0.427547,-0.534151,-0.488662,-0.433413,-0.409950,-0.440809,...,-0.689797,-0.487786,-0.693368,-0.690511,-0.697168,-0.698876,-0.692781,-0.693215,-0.690715,-0.697168
2010-01-06,-0.119448,-0.448459,-0.221735,-0.556089,-0.437237,-0.529564,-0.489938,-0.443103,-0.416323,-0.450499,...,-0.691009,-0.482405,-0.694477,-0.691570,-0.698660,-0.700063,-0.693585,-0.694707,-0.691697,-0.698456
2010-01-07,-0.128070,-0.447880,-0.221425,-0.558309,-0.435383,-0.529490,-0.489612,-0.445840,-0.412941,-0.452471,...,-0.690798,-0.484196,-0.694241,-0.690925,-0.698398,-0.699826,-0.692940,-0.694011,-0.691537,-0.698270
2010-01-08,-0.134691,-0.446343,-0.217845,-0.560598,-0.435631,-0.531524,-0.486035,-0.435631,-0.424155,-0.453229,...,-0.687554,-0.484853,-0.690844,-0.687452,-0.695052,-0.696403,-0.689517,-0.690538,-0.688370,-0.695077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,-0.156359,-0.074674,-0.086014,-0.449274,-0.403146,-0.351636,-0.439087,-0.220555,-0.418522,-0.268797,...,-0.484216,-0.472242,-0.478247,-0.475445,-0.477963,-0.484908,-0.483217,-0.487330,-0.466587,-0.477470
2022-01-25,-0.162735,-0.076463,-0.086007,-0.448464,-0.403419,-0.356466,-0.438157,-0.216178,-0.420216,-0.270766,...,-0.483431,-0.471407,-0.477427,-0.474644,-0.477145,-0.484080,-0.482381,-0.486504,-0.465848,-0.476961
2022-01-26,-0.153456,-0.075840,-0.051442,-0.439711,-0.396293,-0.346918,-0.429913,-0.217623,-0.413775,-0.268534,...,-0.473870,-0.462151,-0.467711,-0.464910,-0.467427,-0.474408,-0.472794,-0.476867,-0.456056,-0.467299


In [84]:
df_close = df.pivot(index=['Trddt'], columns='Stkcd', values=['Clsprc'])
df_close = df_close['Clsprc']
df_close = df_close.reset_index()

In [86]:
df_close.fillna(df_open.mean(), inplace=True)
df_close

Stkcd,Trddt,1,2,4,5,6,7,8,9,10,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
0,2010-01-04,23.71,10.60,19.23949,5.99,11.12,6.93,8.723644,10.84,11.81,...,0.838,8.680,0.695000,0.809000,0.538000,0.480,0.719,0.705,0.798000,0.546
1,2010-01-05,23.30,10.36,19.23949,6.20,10.80,7.19,8.723644,10.56,11.60,...,0.840,8.979,0.704000,0.823000,0.545000,0.485,0.739,0.695,0.810000,0.542
2,2010-01-06,22.90,10.36,19.23949,6.08,10.83,7.19,8.723644,10.55,11.72,...,0.839,8.934,0.705000,0.836000,0.540000,0.482,0.754,0.708,0.809000,0.546
3,2010-01-07,22.65,10.28,19.23949,5.83,10.74,6.98,8.723644,10.75,11.26,...,0.822,8.781,0.693000,0.826000,0.524000,0.476,0.745,0.705,0.793000,0.528
4,2010-01-08,22.60,10.35,19.23949,5.89,11.00,7.03,8.723644,11.27,11.46,...,0.828,8.900,0.695000,0.832000,0.530000,0.481,0.750,0.714,0.792000,0.528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2931,2022-01-24,17.20,21.70,21.45000,2.12,4.46,6.92,2.670000,14.35,3.61,...,0.278,0.906,0.592559,0.738369,0.607353,0.244,0.334,0.118,1.199219,0.621
2932,2022-01-25,16.85,20.82,20.24000,2.05,4.29,6.82,2.520000,13.60,3.39,...,0.271,0.882,0.592559,0.738369,0.607353,0.243,0.329,0.116,1.199219,0.611
2933,2022-01-26,16.65,20.73,22.26000,2.06,4.33,6.48,2.590000,13.35,3.43,...,0.271,0.889,0.592559,0.738369,0.607353,0.247,0.329,0.117,1.199219,0.614
2934,2022-01-27,16.30,20.44,20.40000,2.08,4.24,6.16,2.520000,12.85,3.34,...,0.271,0.886,0.592559,0.738369,0.607353,0.245,0.325,0.111,1.199219,0.612


In [94]:
df_close = df_close.set_index('Trddt')

In [96]:
df_close.index

Index(['2010-01-04', '2010-01-05', '2010-01-06', '2010-01-07', '2010-01-08',
       '2010-01-11', '2010-01-12', '2010-01-13', '2010-01-14', '2010-01-15',
       ...
       '2022-01-17', '2022-01-18', '2022-01-19', '2022-01-20', '2022-01-21',
       '2022-01-24', '2022-01-25', '2022-01-26', '2022-01-27', '2022-01-28'],
      dtype='object', name='Trddt', length=2936)

In [99]:
df_return = df_close / df_close.shift(1) - 1
df_return

Stkcd,1,2,4,5,6,7,8,9,10,11,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
Trddt,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,-0.017292,-0.022642,0.000000,0.035058,-0.028777,0.037518,0.000000,-0.025830,-0.017782,-0.031015,...,0.002387,0.034447,0.012950,0.017305,0.013011,0.010417,0.027816,-0.014184,0.015038,-0.007326
2010-01-06,-0.017167,0.000000,0.000000,-0.019355,0.002778,0.000000,0.000000,-0.000947,0.010345,0.002910,...,-0.001190,-0.005012,0.001420,0.015796,-0.009174,-0.006186,0.020298,0.018705,-0.001235,0.007380
2010-01-07,-0.010917,-0.007722,0.000000,-0.041118,-0.008310,-0.029207,0.000000,0.018957,-0.039249,-0.023211,...,-0.020262,-0.017126,-0.017021,-0.011962,-0.029630,-0.012448,-0.011936,-0.004237,-0.019778,-0.032967
2010-01-08,-0.002208,0.006809,0.000000,0.010292,0.024209,0.007163,0.000000,0.048372,0.017762,0.009901,...,0.007299,0.013552,0.002886,0.007264,0.011450,0.010504,0.006711,0.012766,-0.001261,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,-0.008646,0.002773,0.015625,0.004739,-0.011086,0.011696,0.011364,0.019900,-0.019022,-0.007819,...,-0.021127,0.001105,0.000000,0.000000,0.000000,-0.016129,-0.011834,-0.016667,0.000000,-0.026646
2022-01-25,-0.020349,-0.040553,-0.056410,-0.033019,-0.038117,-0.014451,-0.056180,-0.052265,-0.060942,-0.044658,...,-0.025180,-0.026490,0.000000,0.000000,0.000000,-0.004098,-0.014970,-0.016949,0.000000,-0.016103
2022-01-26,-0.011869,-0.004323,0.099802,0.004878,0.009324,-0.049853,0.027778,-0.018382,0.011799,0.024748,...,0.000000,0.007937,0.000000,0.000000,0.000000,0.016461,0.000000,0.008621,0.000000,0.004910


In [103]:
import statsmodels.api as sm 
def compute_IC_t(df,return_nextday):#df为因子值矩阵
    
    #行列对齐
    return_nextday=return_nextday[df.columns.tolist()]
    return_nextday=return_nextday.loc[df.index.tolist()]
    
    #建立新的dataframe，用于存储结果
    result=pd.DataFrame(np.nan,index=df.index,columns=["factor_return","tvalue","IC"])

    #T+1期收益率对T期因子值逐行做回归
    for date in df.index.tolist():
        x=df.loc[date]
        y=return_nextday.loc[date]
        model=sm.OLS(y,x).fit()
        result.loc[date,"factor_return"]=model.params[date]
        result.loc[date,"tvalue"]=model.tvalues[date]
        result.loc[date,"IC"]=return_nextday.loc[date].corr(df.loc[date])
    
    return result

In [110]:
df_result = compute_IC_t(df_open_std.shift(2)[2:], df_return)

In [116]:
df_result

,factor_return,tvalue,IC
Trddt,,,
2010-01-06,-0.000498,-0.565485,-0.008143
2010-01-07,0.002502,3.371227,0.048870
2010-01-08,-0.002808,-2.606860,-0.037597
2010-01-11,-0.000523,-0.411956,-0.005944
2010-01-12,-0.003139,-2.624245,-0.038071
...,...,...,...
2022-01-24,0.002951,6.017240,0.086530
2022-01-25,0.003078,4.175669,0.091840
2022-01-26,-0.001160,-2.864023,-0.042189


In [122]:
df_result[df_result['factor_return'] > 0]

,factor_return,tvalue,IC
Trddt,,,
2010-01-07,0.002502,3.371227,0.048870
2010-01-13,0.001011,0.949076,0.013670
2010-01-20,0.004550,2.574909,0.037132
2010-01-22,0.001711,1.960403,0.028514
2010-01-25,0.001705,2.281216,0.032972
...,...,...,...
2022-01-18,0.001407,2.722897,0.041706
2022-01-20,0.000763,1.280887,0.021988
2022-01-24,0.002951,6.017240,0.086530


### 因子

In [123]:
df

,Stkcd,Trddt,Opnprc,Hiprc,Loprc,Clsprc,Dnshrtrd,Dnvaltrd,Dsmvosd,Dsmvtll,Dretwd,Dretnd,Adjprcwd,Adjprcnd,Markettype,Capchgdt,Trdsta,IndustryCode,IndustryName
0,1,2010-01-04,24.520,24.580,23.680,23.710,24192276,5.802495e+08,69330749.18,73629834.50,-0.027082,-0.027082,818.361203,710.377219,4,2009-10-15,1,480101.0,����
1,1,2010-01-05,23.750,23.900,22.750,23.300,55649982,1.293477e+09,68131862.33,72356606.65,-0.017292,-0.017292,804.209870,698.093176,4,2009-10-15,1,480101.0,����
2,1,2010-01-06,23.250,23.250,22.720,22.900,41214313,9.444537e+08,66962216.62,71114433.15,-0.017167,-0.017167,790.403692,686.108744,4,2009-10-15,1,480101.0,����
3,1,2010-01-07,22.900,23.050,22.400,22.650,35533685,8.041663e+08,66231188.06,70338074.71,-0.010917,-0.010917,781.774831,678.618474,4,2009-10-15,1,480101.0,����
4,1,2010-01-08,22.500,22.750,22.350,22.600,28854306,6.506674e+08,66084982.34,70182803.02,-0.002208,-0.002208,780.049059,677.120420,4,2009-10-15,1,480101.0,����
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8672604,900957,2022-01-24,0.633,0.638,0.615,0.621,479100,2.998840e+05,114264.00,216729.00,-0.026646,-0.026646,0.625923,0.620999,2,2000-08-03,1,620201.0,װ��װ��
8672605,900957,2022-01-25,0.617,0.620,0.607,0.611,372779,2.281030e+05,112424.00,213239.00,-0.016103,-0.016103,0.615844,0.610999,2,2000-08-03,1,620201.0,װ��װ��
8672606,900957,2022-01-26,0.614,0.617,0.610,0.614,222600,1.362910e+05,112976.00,214286.00,0.004910,0.004910,0.618868,0.613999,2,2000-08-03,1,620201.0,װ��װ��
8672607,900957,2022-01-27,0.610,0.613,0.605,0.612,245100,1.487580e+05,112608.00,213588.00,-0.003257,-0.003257,0.616852,0.611999,2,2000-08-03,1,620201.0,װ��װ��


In [129]:
get_data('Hiprc')

Stkcd,1,2,4,5,6,7,8,9,10,11,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
Trddt,,,,,,,,,,,,,,,,,,,,,
2010-01-04,24.58,10.87,19.702147,6.05,11.35,7.09,8.902363,11.03,12.08,10.79,...,0.853,8.816,0.708000,0.837000,0.541000,0.484,0.727,0.716,0.818000,0.559
2010-01-05,23.90,10.52,19.702147,6.32,11.18,7.19,8.902363,10.95,11.85,10.71,...,0.845,9.018,0.707000,0.890000,0.548000,0.486,0.746,0.708,0.817000,0.548
2010-01-06,23.25,10.51,19.702147,6.28,11.04,7.30,8.902363,10.75,11.73,10.52,...,0.852,9.022,0.713000,0.844000,0.549000,0.489,0.775,0.713,0.814000,0.549
2010-01-07,23.05,10.43,19.702147,6.07,10.96,7.24,8.902363,11.09,11.77,10.41,...,0.840,9.000,0.706000,0.838000,0.541000,0.485,0.765,0.716,0.810000,0.547
2010-01-08,22.75,10.38,19.702147,5.93,11.00,7.05,8.902363,11.32,11.46,10.20,...,0.829,8.900,0.699000,0.834000,0.530000,0.482,0.755,0.715,0.794000,0.531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,17.38,22.00,22.580000,2.15,4.54,7.18,2.700000,14.45,3.71,11.65,...,0.282,0.912,0.598721,0.751285,0.618736,0.247,0.342,0.120,1.216933,0.638
2022-01-25,17.08,21.68,22.100000,2.13,4.48,6.95,2.690000,14.44,3.64,11.49,...,0.278,0.909,0.598721,0.751285,0.618736,0.246,0.334,0.119,1.216933,0.620
2022-01-26,17.10,21.11,22.260000,2.09,4.38,6.88,2.610000,13.80,3.45,11.24,...,0.274,0.890,0.598721,0.751285,0.618736,0.247,0.330,0.118,1.216933,0.617
